# Coursera Capston - Applied Data Science

## Gary S. Netherton - June 30, 2020

### This notebook will be used for the capstone project for the Applied Data Science coursework.

In [4]:
import pandas as pd
import numpy as np

In [5]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


### Purpose: 
> I have the opportunity to relocate.  My choice, however, will revolve around the selection of restaurants and the performing arts.  I have lived in
> places such as Chicago that have a plethora of food options.  I have also lived places where my options were limited to Subway, McDonald's, and a couple of 
> teriyaki joints.  As I want this to be my last move - I am nearing retirement - I want to find a comfortable, compatible city to match my wants and needs.

> Additionally, I am a museum and play/performance junky.  I really enjoy walking through museums and attending live performances..
> I am currently reviewing job offers from two firms - one is located in Temecula, CA and one is located in Portland, OR.

###          Target Audience(s):
> 1. Me
> 2. Families with children
> 3. Families with cultural backgrounds

> *The Foursquare API will enable me to research both dining establishments as well as medical facilities.  I will review both the number of both as well as the*
> *ratings provided by users.  Due to recent changes in the Foursquare API for security purposes, I will not dig into specific users and their comments.*

In [6]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

print('Libraries imported.')
import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

# Let me know when the libraries are loaded.

print('Libraries imported.')


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\gnetherton\Anaconda3

  added / updated specs:
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    certifi-2019.11.28         |           py37_0         148 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         835 KB

The following NEW packages will be INSTALLED:

  altair             conda-forge/noarch::altair-4.1.0-py_1
  vincent            conda-forge/noarch::vince

### Data Collection

In [7]:
# Use pandas to import data from both California and Oregon
df_All = pd.read_csv('zip_code_database_CA.csv')

df_All['latitude']=pd.to_numeric(df_All['latitude'])
df_All['longitude']=pd.to_numeric(df_All['longitude'])
df_All['irs_estimated_population_2015']=pd.to_numeric(df_All['irs_estimated_population_2015'])

#df_All.head()
df_CA = df_All.loc[df_All['area_codes']=='951',:]
print(df_CA.shape)
#df_OR = df_All.loc[(df_All['area_codes']=='503' & df_All['area_codes']=='971'),:]
df_OR = df_All.loc[(df_All['area_codes']=='503') | (df_All['area_codes']=='971'), :]
print(df_OR.shape)


(48, 15)
(118, 15)


In [8]:
df_CA.head()

zip      type  decommissioned primary_city acceptable_cities  \
38625  91752  STANDARD               0    Mira Loma          Eastvale   
38863  92220  STANDARD               0      Banning               NaN   
39012  92502    PO BOX               0    Riverside               NaN   
39019  92509  STANDARD               0    Riverside     Jurupa Valley   
39020  92513    PO BOX               0    Riverside               NaN   

      unacceptable_cities state            county             timezone  \
38625                 NaN    CA  Riverside County  America/Los_Angeles   
38863                 NaN    CA  Riverside County  America/Los_Angeles   
39012                 NaN    CA  Riverside County  America/Los_Angeles   
39019            Rubidoux    CA  Riverside County  America/Los_Angeles   
39020                 NaN    CA  Riverside County  America/Los_Angeles   

      area_codes world_region country  latitude  longitude  \
38625        951          NaN      US     33.99    -117.53   
38863        951          NaN      US     33.95    -116.83   
39012        951          NaN      US     33.98    -117.37   
39019        951          NaN      US     33.99    -117.44   
39020        951          NaN      US     33.91    -117.46   

       irs_estimated_population_2015  
38625                          30490  
38863                          26920  
39012                            979  
39019                          72670  
39020                           1241

In [9]:
df_OR.head()

zip      type  decommissioned primary_city acceptable_cities  \
41134  97002  STANDARD               0       Aurora               NaN   
41136  97004  STANDARD               0  Beavercreek               NaN   
41141  97009  STANDARD               0       Boring          Damascus   
41142  97010    PO BOX               0  Bridal Veil               NaN   
41144  97013  STANDARD               0        Canby               NaN   

      unacceptable_cities state            county             timezone  \
41134          Butteville    OR     Marion County  America/Los_Angeles   
41136                 NaN    OR  Clackamas County  America/Los_Angeles   
41141                 NaN    OR  Clackamas County  America/Los_Angeles   
41142                 NaN    OR  Multnomah County  America/Los_Angeles   
41144              Barlow    OR  Clackamas County  America/Los_Angeles   

      area_codes world_region country  latitude  longitude  \
41134        503          NaN      US     45.23    -122.80   
41136        503          NaN      US     45.25    -122.46   
41141        971          NaN      US     45.42    -122.37   
41142        971          NaN      US     45.58    -122.11   
41144        503          NaN      US     45.21    -122.66   

       irs_estimated_population_2015  
41134                           5420  
41136                           4270  
41141                           8140  
41142                             49  
41144                          22350

In [10]:
# Oregon Latitude 44.1156, Longitude -120.5148
# California Latitude 37.2461, Longitude -119.6104

In [11]:
# Setup the Foursquare API information

CLIENT_ID = 'BRBDGCKR00L1JCYN5RD3ZDJ2BNOJ5YOVAN3F2KZNVVUVB3VW' # your Foursquare ID
CLIENT_SECRET = '5N1FQNDEEVF0LSK4A1SFM41P4KFSGECE1T23QNK20K1SD4PR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: BRBDGCKR00L1JCYN5RD3ZDJ2BNOJ5YOVAN3F2KZNVVUVB3VW
CLIENT_SECRET:5N1FQNDEEVF0LSK4A1SFM41P4KFSGECE1T23QNK20K1SD4PR


In [12]:
# create map of Temecula using latitude and longitude values

map_temecula = folium.Map(location=[33.4934, -117.14879], zoom_start=10)

# add markers to map
for lat, lng, primCity, popul in zip(df_CA['latitude'], df_CA['longitude'], df_CA['primary_city'], df_CA['irs_estimated_population_2015']):
    label = '{}, {}'.format(primCity, popul)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_temecula)  
    
map_temecula

In [13]:
# create map of Portland using latitude and longitude values

map_portland = folium.Map(location=[45.51179, -122.67563], zoom_start=10)

# add markers to map
for lat, lng, primCity, popul in zip(df_OR['latitude'], df_OR['longitude'], df_OR['primary_city'], df_OR['irs_estimated_population_2015']):
    label = '{}, {}'.format(primCity, popul)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_portland)  
    
map_portland

In [14]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [15]:
# Create function to collect venue information from all neighborhoods in Temecula & Portland
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# Collect the information from Foursquare for Temecula neighborhoods
temecula_venues = getNearbyVenues(df_CA['primary_city'],
                                   df_CA['latitude'],
                                   df_CA['longitude']
                                  )

Mira Loma
Banning
Riverside
Riverside
Riverside
Riverside
Riverside
Riverside
Riverside
March Air Reserve Base
Riverside
Riverside
Riverside
Lake Elsinore
Lake Elsinore
Lake Elsinore
Aguanga
Anza
Hemet
Homeland
Idyllwild
Moreno Valley
Moreno Valley
Moreno Valley
Moreno Valley
Moreno Valley
Moreno Valley
Moreno Valley
Murrieta
Murrieta
Murrieta
Nuevo
Perris
Perris
Perris
San Jacinto
San Jacinto
Menifee
Sun City
Sun City
Sun City
Temecula
Temecula
Wildomar
Perris
Corona
Corona
Corona


In [17]:
# Collect the information from Foursquare for Portland neighborhoods
portland_venues = getNearbyVenues(df_OR['primary_city'],
                                   df_OR['latitude'],
                                   df_OR['longitude']
                                  )

Aurora
Beavercreek
Boring
Bridal Veil
Canby
Clackamas
Colton
Columbia City
Corbett
Donald
Eagle Creek
Estacada
Fairview
Gervais
Government Camp
Lake Oswego
Lake Oswego
Marylhurst
Molalla
Mulino
Warren
Deer Island
Sandy
Scappoose
Troutdale
Beaverton
Beaverton
Beaverton
Beaverton
Happy Valley
Damascus
Arch Cape
Banks
Beaver
Buxton
Cannon Beach
Carlton
Dundee
Forest Grove
Gales Creek
Garibaldi
Hammond
Hebo
Hillsboro
Manning
Lafayette
Manzanita
Oceanside
Pacific City
Sherwood
Netarts
Timber
Tolovana Park
Warrenton
Wheeler
Neskowin
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Portland
Salem
Salem
Keizer
Salem
Salem
Salem
Salem
Salem
Salem
Salem
Salem
Dallas
Detroit
Grand Ronde
Inde

In [18]:
portland_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0       Aurora                  45.23                 -122.80   
1       Aurora                  45.23                 -122.80   
2       Aurora                  45.23                 -122.80   
3       Aurora                  45.23                 -122.80   
4  Bridal Veil                  45.58                 -122.11   

                          Venue  Venue Latitude  Venue Longitude  \
0       Rome Transportation LLC       45.231671      -122.801974   
1      XFINITY Store by Comcast       45.228450      -122.803878   
2   Carpet Cleaning Wilsonville       45.229599      -122.805369   
3       Sunrise Custom Painting       45.225655      -122.800477   
4  Cousin's Restaurant & Saloon       45.579640      -122.111006   

        Venue Category  
0             Building  
1     Business Service  
2         Carpet Store  
3      Paintball Field  
4  American Restaurant

In [19]:
temecula_venues.head()

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
0    Mira Loma                  33.99                 -117.53   
1      Banning                  33.95                 -116.83   
2    Riverside                  33.98                 -117.37   
3    Riverside                  33.98                 -117.37   
4    Riverside                  33.98                 -117.37   

                            Venue  Venue Latitude  Venue Longitude  \
0               Party Productions       33.987651      -117.534519   
1                    Malki Museum       33.949280      -116.825279   
2       The Old Spaghetti Factory       33.980494      -117.367675   
3  Riverside Municipal Auditorium       33.982062      -117.370789   
4              Romano's Riverside       33.981286      -117.372714   

          Venue Category  
0                   Food  
1                 Museum  
2     Italian Restaurant  
3  Performing Arts Venue  
4     Italian Restaurant

In [20]:
# Number of venues returned for each neighborhood
temecula_venues.groupby('Neighborhood').count()

Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                                   
Banning                                     1                       1      1   
Corona                                      8                       8      8   
Hemet                                       3                       3      3   
Homeland                                    1                       1      1   
Idyllwild                                   1                       1      1   
Lake Elsinore                              14                      14     14   
March Air Reserve Base                      2                       2      2   
Menifee                                     2                       2      2   
Mira Loma                                   1                       1      1   
Moreno Valley                              36                      36     36   
Murrieta                                   14                      14     14   
Perris                                     24                      24     24   
Riverside                                 173                     173    173   
San Jacinto                                 3                       3      3   
Sun City                                    4                       4      4   
Temecula                                   52                      52     52   

                        Venue Latitude  Venue Longitude  Venue Category  
Neighborhood                                                             
Banning                              1                1               1  
Corona                               8                8               8  
Hemet                                3                3               3  
Homeland                             1                1               1  
Idyllwild                            1                1               1  
Lake Elsinore                       14               14              14  
March Air Reserve Base               2                2               2  
Menifee                              2                2               2  
Mira Loma                            1                1               1  
Moreno Valley                       36               36              36  
Murrieta                            14               14              14  
Perris                              24               24              24  
Riverside                          173              173             173  
San Jacinto                          3                3               3  
Sun City                             4                4               4  
Temecula                            52               52              52

In [21]:
# Number of venues returned for each neighborhood
portland_venues.groupby('Neighborhood').count()

Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                          
Arch Cape                          1                       1      1   
Aurora                             4                       4      4   
Beaverton                        207                     207    207   
Bridal Veil                        1                       1      1   
Canby                              1                       1      1   
Cannon Beach                      50                      50     50   
Clackamas                          4                       4      4   
Columbia City                      3                       3      3   
Damascus                           9                       9      9   
Fairview                           3                       3      3   
Forest Grove                       1                       1      1   
Grand Ronde                        1                       1      1   
Happy Valley                       2                       2      2   
Hillsboro                          1                       1      1   
Keizer                             2                       2      2   
Lafayette                          4                       4      4   
Lake Oswego                       29                      29     29   
Manzanita                          1                       1      1   
Marylhurst                        18                      18     18   
Mehama                             2                       2      2   
Mulino                             1                       1      1   
Netarts                            3                       3      3   
Oceanside                          1                       1      1   
Portland                         974                     974    974   
Rickreall                          1                       1      1   
Salem                             50                      50     50   
Sherwood                           4                       4      4   
Timber                             1                       1      1   
Troutdale                          2                       2      2   
Wheeler                            9                       9      9   

               Venue Latitude  Venue Longitude  Venue Category  
Neighborhood                                                    
Arch Cape                   1                1               1  
Aurora                      4                4               4  
Beaverton                 207              207             207  
Bridal Veil                 1                1               1  
Canby                       1                1               1  
Cannon Beach               50               50              50  
Clackamas                   4                4               4  
Columbia City               3                3               3  
Damascus                    9                9               9  
Fairview                    3                3               3  
Forest Grove                1                1               1  
Grand Ronde                 1                1               1  
Happy Valley                2                2               2  
Hillsboro                   1                1               1  
Keizer                      2                2               2  
Lafayette                   4                4               4  
Lake Oswego                29               29              29  
Manzanita                   1                1               1  
Marylhurst                 18               18              18  
Mehama                      2                2               2  
Mulino                      1                1               1  
Netarts                     3                3               3  
Oceanside                   1                1               1  
Portland                  974              974             974  
Rickreall                   1                1      

In [22]:
print('There are {} uniques categories for Temecula.'.format(len(temecula_venues['Venue Category'].unique())))
print('There are {} uniques categories for Portland.'.format(len(portland_venues['Venue Category'].unique())))


There are 121 uniques categories for Temecula.
There are 230 uniques categories for Portland.


In [23]:
# one hot encoding - Temecula
temecula_onehot = pd.get_dummies(temecula_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
temecula_onehot['Neighborhood'] = temecula_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [temecula_onehot.columns[-1]] + list(temecula_onehot.columns[:-1])
temecula_onehot = temecula_onehot[fixed_columns]
print(temecula_onehot.shape)
temecula_onehot.head()

(339, 122)


Neighborhood  ATM  Accessories Store  Advertising Agency  \
0    Mira Loma    0                  0                   0   
1      Banning    0                  0                   0   
2    Riverside    0                  0                   0   
3    Riverside    0                  0                   0   
4    Riverside    0                  0                   0   

   American Restaurant  Antique Shop  Art Gallery  Art Museum  \
0                    0             0            0           0   
1                    0             0            0           0   
2                    0             0            0           0   
3                    0             0            0           0   
4                    0             0            0           0   

   Arts & Crafts Store  Asian Restaurant  Athletics & Sports  Automotive Shop  \
0                    0                 0                   0                0   
1                    0                 0                   0                0   
2                    0                 0                   0                0   
3                    0                 0                   0                0   
4                    0                 0                   0                0   

   Bakery  Bank  Bar  Baseball Field  Beer Garden  Bistro  Breakfast Spot  \
0       0     0    0               0            0       0               0   
1       0     0    0               0            0       0               0   
2       0     0    0               0            0       0               0   
3       0     0    0               0            0       0               0   
4       0     0    0               0            0       0               0   

   Bridal Shop  Burger Joint  Café  Cajun / Creole Restaurant  Candy Store  \
0            0             0     0                          0            0   
1            0             0     0                          0            0   
2            0             0     0                          0            0   
3            0             0     0                          0            0   
4            0             0     0                          0            0   

   Chinese Restaurant  Chocolate Shop  Clothing Store  Cocktail Bar  \
0                   0               0               0             0   
1                   0               0               0             0   
2                   0               0               0             0   
3                   0               0               0             0   
4                   0               0               0             0   

   Coffee Shop  Comic Shop  Construction & Landscaping  Convenience Store  \
0            0           0                           0                  0   
1            0           0                           0                  0   
2            0           0                           0                  0   
3            0           0                           0                  0   
4            0           0                           0                  0   

   Cosmetics Shop  Deli / Bodega  Department Store  Dessert Shop  Diner  \
0               0              0                 0             0      0   
1               0              0                 0             0      0   
2               0              0                 0             0      0   
3               0              0                 0             0      0   
4               0              0                 0             0      0   

   Discount Store  Donut Shop  Farmers Market  Fast Food Restaurant  \
0               0           0               0                     0   
1               0           0               0                     0   
2               0           0               0                     0   
3               0           0               0                     0   
4               0           0               0                     0   

   Filipino Restaurant  Flea Market  Food  Fried Ch

In [24]:
# one hot encoding - Portland
portland_onehot = pd.get_dummies(portland_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
portland_onehot['Neighborhood'] = portland_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [portland_onehot.columns[-1]] + list(portland_onehot.columns[:-1])
portland_onehot = portland_onehot[fixed_columns]
print(portland_onehot.shape)
portland_onehot.head()

(1390, 231)


Neighborhood  ATM  Airport  Airport Terminal  American Restaurant  \
0       Aurora    0        0                 0                    0   
1       Aurora    0        0                 0                    0   
2       Aurora    0        0                 0                    0   
3       Aurora    0        0                 0                    0   
4  Bridal Veil    0        0                 0                    1   

   Antique Shop  Art Gallery  Arts & Crafts Store  Asian Restaurant  \
0             0            0                    0                 0   
1             0            0                    0                 0   
2             0            0                    0                 0   
3             0            0                    0                 0   
4             0            0                    0                 0   

   Athletics & Sports  Automotive Shop  Bagel Shop  Bakery  Bank  Bar  \
0                   0                0           0       0     0    0   
1                   0                0           0       0     0    0   
2                   0                0           0       0     0    0   
3                   0                0           0       0     0    0   
4                   0                0           0       0     0    0   

   Baseball Field  Baseball Stadium  Basketball Court  Beach  Bed & Breakfast  \
0               0                 0                 0      0                0   
1               0                 0                 0      0                0   
2               0                 0                 0      0                0   
3               0                 0                 0      0                0   
4               0                 0                 0      0                0   

   Beer Bar  Beer Garden  Beer Store  Bike Shop  Bike Trail  Bistro  \
0         0            0           0          0           0       0   
1         0            0           0          0           0       0   
2         0            0           0          0           0       0   
3         0            0           0          0           0       0   
4         0            0           0          0           0       0   

   Boat or Ferry  Bookstore  Boutique  Brazilian Restaurant  Breakfast Spot  \
0              0          0         0                     0               0   
1              0          0         0                     0               0   
2              0          0         0                     0               0   
3              0          0         0                     0               0   
4              0          0         0                     0               0   

   Brewery  Bubble Tea Shop  Building  Burger Joint  Burrito Place  \
0        0                0         1             0              0   
1        0                0         0             0              0   
2        0                0         0             0              0   
3        0                0         0             0              0   
4        0                0         0             0              0   

   Bus Station  Bus Stop  Business Service  Café  Cajun / Creole Restaurant  \
0            0         0                 0     0                          0   
1            0         0                 1     0                          0   
2            0         0                 0     0                          0   
3            0         0                 0     0                          0   
4            0         0                 0     0                          0   

   Campground  Candy Store  Caribbean Restaurant  Carpet Store  Cheese Shop  \
0           0            0                     0             0            0   
1           0            0                     0             0            0   
2           0            0                     0             1            0   
3           0            0                     0             0            0   
4           0            0   

In [25]:
temecula_grouped = temecula_onehot.groupby('Neighborhood').mean().reset_index()
print('Temecula:')
print(temecula_grouped.shape)
portland_grouped = portland_onehot.groupby('Neighborhood').mean().reset_index()
print("Portland:")
print(portland_grouped.shape)

Temecula:
(16, 122)
Portland:
(30, 231)


In [26]:
num_top_venues = 5

for hood in temecula_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = temecula_grouped[temecula_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Banning----
                   venue  freq
0                 Museum   1.0
1                    ATM   0.0
2              Hotel Bar   0.0
3            Pizza Place   0.0
4  Performing Arts Venue   0.0


----Corona----
               venue  freq
0  Convenience Store  0.50
1              Diner  0.25
2               Park  0.25
3                Pub  0.00
4        Pizza Place  0.00


----Hemet----
            venue  freq
0  Sandwich Place  0.33
1   Grocery Store  0.33
2    Liquor Store  0.33
3             ATM  0.00
4  Massage Studio  0.00


----Homeland----
                   venue  freq
0           Home Service   1.0
1                    ATM   0.0
2                 Market   0.0
3  Performing Arts Venue   0.0
4                   Park   0.0


----Idyllwild----
                   venue  freq
0                    Spa   1.0
1                    ATM   0.0
2                   Pool   0.0
3  Performing Arts Venue   0.0
4                   Park   0.0


----Lake Elsinore----
                       v

In [27]:

for hood in portland_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp1 = portland_grouped[portland_grouped['Neighborhood'] == hood].T.reset_index()
    temp1.columns = ['venue','freq']
    temp1 = temp1.iloc[1:]
    temp1['freq'] = temp1['freq'].astype(float)
    temp1 = temp1.round({'freq': 2})
    print(temp1.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arch Cape----
         venue  freq
0       Tunnel   1.0
1          ATM   0.0
2  Pastry Shop   0.0
3  Music Store   0.0
4  Music Venue   0.0


----Aurora----
              venue  freq
0          Building  0.25
1  Business Service  0.25
2   Paintball Field  0.25
3      Carpet Store  0.25
4               ATM  0.00


----Beaverton----
                venue  freq
0   Korean Restaurant  0.06
1         Coffee Shop  0.05
2  Mexican Restaurant  0.04
3      Sandwich Place  0.04
4       Grocery Store  0.04


----Bridal Veil----
                   venue  freq
0    American Restaurant   1.0
1                    ATM   0.0
2  Performing Arts Venue   0.0
3            Music Venue   0.0
4             Nail Salon   0.0


----Canby----
                     venue  freq
0              Cheese Shop   1.0
1    Performing Arts Venue   0.0
2              Music Venue   0.0
3               Nail Salon   0.0
4  New American Restaurant   0.0


----Cannon Beach----
                 venue  freq
0  American Restauran

In [28]:

# Insert the most common venues data into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [29]:

# The following code creates a new dataframe that will display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = temecula_grouped['Neighborhood']

for ind in np.arange(temecula_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(temecula_grouped.iloc[ind, :], num_top_venues)

print('For Temecula: ')
neighborhoods_venues_sorted.head(10)

For Temecula: 


Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0                 Banning                Museum         Women's Store   
1                  Corona     Convenience Store                  Park   
2                   Hemet          Liquor Store         Grocery Store   
3                Homeland          Home Service         Hot Dog Joint   
4               Idyllwild                   Spa         Women's Store   
5           Lake Elsinore        Sandwich Place  Fast Food Restaurant   
6  March Air Reserve Base           Coffee Shop  Fast Food Restaurant   
7                 Menifee     Mobile Phone Shop         Garden Center   
8               Mira Loma                  Food         Women's Store   
9           Moreno Valley    Mexican Restaurant   Fried Chicken Joint   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0     Convenience Store         Deli / Bodega      Department Store   
1                 Diner         Women's Store         Deli / Bodega   
2        Sandwich Place   Fried Chicken Joint        Cosmetics Shop   
3        Cosmetics Shop         Deli / Bodega      Department Store   
4   Fried Chicken Joint        Cosmetics Shop         Deli / Bodega   
5           Pizza Place   Fried Chicken Joint     Convenience Store   
6         Women's Store     Convenience Store         Deli / Bodega   
7         Women's Store   Fried Chicken Joint         Deli / Bodega   
8         Hot Dog Joint        Cosmetics Shop         Deli / Bodega   
9        Ice Cream Shop   Filipino Restaurant  Fast Food Restaurant   

  6th Most Common Venue      7th Most Common Venue      8th Most Common Venue  \
0          Dessert Shop                      Diner             Discount Store   
1      Department Store               Dessert Shop             Discount Store   
2         Deli / Bodega           Department Store               Dessert Shop   
3          Dessert Shop                      Diner             Discount Store   
4      Department Store               Dessert Shop                      Diner   
5                   Bar  Latin American Restaurant  Cajun / Creole Restaurant   
6      Department Store               Dessert Shop                      Diner   
7      Department Store               Dessert Shop                      Diner   
8      Department Store               Dessert Shop                      Diner   
9                Bakery            Automotive Shop              Deli / Bodega   

  9th Most Common Venue 10th Most Common Venue  
0            Donut Shop         Farmers Market  
1            Donut Shop         Farmers Market  
2                 Diner         Discount Store  
3            Donut Shop         Farmers Market  
4        Discount Store             Donut Shop  
5                  Food         Discount Store  
6        Discount Store             Donut Shop  
7        Discount Store             Donut Shop  
8        Discount Store             Donut Shop  
9      Asian Restaurant            Pizza Place

In [30]:
# The following code creates a new dataframe that will display the top 10 venues for each neighborhood
#num_top_venues = 10

indicators1 = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns1 = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns1.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns1.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted1 = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted1['Neighborhood'] = portland_grouped['Neighborhood']

for ind in np.arange(portland_grouped.shape[0]):
    neighborhoods_venues_sorted1.iloc[ind, 1:] = return_most_common_venues(portland_grouped.iloc[ind, :], num_top_venues)

print('For Portland: ')
neighborhoods_venues_sorted1.head(10)

For Portland: 


Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0      Arch Cape                Tunnel           Yoga Studio   
1         Aurora          Carpet Store              Building   
2      Beaverton     Korean Restaurant           Coffee Shop   
3    Bridal Veil   American Restaurant           Yoga Studio   
4          Canby           Cheese Shop           Yoga Studio   
5   Cannon Beach   American Restaurant            Restaurant   
6      Clackamas      Basketball Court        Clothing Store   
7  Columbia City                  Park        History Museum   
8       Damascus         Grocery Store          Home Service   
9       Fairview         Moving Target     Recreation Center   

         3rd Most Common Venue        4th Most Common Venue  \
0                   Donut Shop                   Food Truck   
1             Business Service              Paintball Field   
2               Sandwich Place           Mexican Restaurant   
3  Eastern European Restaurant                       Forest   
4  Eastern European Restaurant                       Forest   
5             Toy / Game Store                  Art Gallery   
6                          Bar                      Brewery   
7                        Beach  Eastern European Restaurant   
8             Business Service               Sandwich Place   
9             Business Service                  Yoga Studio   

         5th Most Common Venue 6th Most Common Venue   7th Most Common Venue  \
0                   Food Court                  Food       Fondue Restaurant   
1                  Yoga Studio            Food Court                    Food   
2                Grocery Store                Bakery             Wings Joint   
3                   Food Truck            Food Court                    Food   
4                   Food Truck            Food Court                    Food   
5                       Bakery                  Café  Furniture / Home Store   
6                  Yoga Studio              Fountain                  Forest   
7                   Food Truck            Food Court                    Food   
8                  Coffee Shop        Ice Cream Shop                     Pub   
9  Eastern European Restaurant            Food Court                    Food   

        8th Most Common Venue        9th Most Common Venue  \
0                 Flower Shop            Fish & Chips Shop   
1           Fondue Restaurant                  Flower Shop   
2          Chinese Restaurant             Sushi Restaurant   
3           Fondue Restaurant                  Flower Shop   
4           Fondue Restaurant                  Flower Shop   
5                 Candy Store           Seafood Restaurant   
6                  Food Truck                   Food Court   
7           Fondue Restaurant                  Flower Shop   
8  Construction & Landscaping  Eastern European Restaurant   
9           Fondue Restaurant                  Flower Shop   

      10th Most Common Venue  
0       Fast Food Restaurant  
1          Fish & Chips Shop  
2  Middle Eastern Restaurant  
3          Fish & Chips Shop  
4          Fish & Chips Shop  
5                Pizza Place  
6                       Food  
7          Fish & Chips Shop  
8                       Food  
9          Fish & Chips Shop

In [31]:

# set number of clusters
kclusters = 5

temecula_grouped_clustering = temecula_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(temecula_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([2, 1, 1, 3, 4, 1, 1, 1, 0, 1])

In [32]:
# add clustering labels for new dataframe to include the cluster and its top 10 venues for Temecula
# currently commented out since I have run this cell so many times
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
# neighborhoods_venues_sorted.head()
temecula_merged = df_CA[['primary_city', 'latitude', 'longitude']]
temecula_merged.rename(columns = {'primary_city':'Neighborhood'}, inplace = True)
temecula_merged.head()


# merge temecula_grouped with df_CA to add latitude/longitude for each neighborhood
neighborhoods_venues_sorted = neighborhoods_venues_sorted.join(temecula_merged.set_index('Neighborhood'), on='Neighborhood')
neighborhoods_venues_sorted.head()

C:\Users\gnetherton\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Cluster Labels Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0               2      Banning                Museum         Women's Store   
1               1       Corona     Convenience Store                  Park   
1               1       Corona     Convenience Store                  Park   
1               1       Corona     Convenience Store                  Park   
2               1        Hemet          Liquor Store         Grocery Store   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0     Convenience Store         Deli / Bodega      Department Store   
1                 Diner         Women's Store         Deli / Bodega   
1                 Diner         Women's Store         Deli / Bodega   
1                 Diner         Women's Store         Deli / Bodega   
2        Sandwich Place   Fried Chicken Joint        Cosmetics Shop   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0          Dessert Shop                 Diner        Discount Store   
1      Department Store          Dessert Shop        Discount Store   
1      Department Store          Dessert Shop        Discount Store   
1      Department Store          Dessert Shop        Discount Store   
2         Deli / Bodega      Department Store          Dessert Shop   

  9th Most Common Venue 10th Most Common Venue  latitude  longitude  
0            Donut Shop         Farmers Market     33.95    -116.83  
1            Donut Shop         Farmers Market     33.87    -117.56  
1            Donut Shop         Farmers Market     33.87    -117.56  
1            Donut Shop         Farmers Market     33.75    -117.48  
2                 Diner         Discount Store     33.74    -116.97

In [33]:


portland_grouped_clustering = portland_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans2 = KMeans(n_clusters=kclusters, random_state=0).fit(portland_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans2.labels_[0:10]

array([4, 0, 0, 2, 0, 0, 0, 0, 0, 0])

In [34]:
# add clustering labels for new dataframe to include the cluster and its top 10 venues for Portland
# currently commented out since I have run this cell so many times
neighborhoods_venues_sorted1.insert(0, 'Cluster Labels', kmeans2.labels_)
# neighborhoods_venues_sorted1.head()
portland_merged = df_OR[['primary_city', 'latitude', 'longitude']]
portland_merged.rename(columns = {'primary_city':'Neighborhood'}, inplace = True)
#portland_merged.head()


# merge portland_merged with df_CA to add latitude/longitude for each neighborhood
neighborhoods_venues_sorted1 = neighborhoods_venues_sorted1.join(portland_merged.set_index('Neighborhood'), on='Neighborhood')
neighborhoods_venues_sorted1.head()

C:\Users\gnetherton\Anaconda3\lib\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Cluster Labels Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0               4    Arch Cape                Tunnel           Yoga Studio   
1               0       Aurora          Carpet Store              Building   
2               0    Beaverton     Korean Restaurant           Coffee Shop   
2               0    Beaverton     Korean Restaurant           Coffee Shop   
2               0    Beaverton     Korean Restaurant           Coffee Shop   

  3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
0            Donut Shop            Food Truck            Food Court   
1      Business Service       Paintball Field           Yoga Studio   
2        Sandwich Place    Mexican Restaurant         Grocery Store   
2        Sandwich Place    Mexican Restaurant         Grocery Store   
2        Sandwich Place    Mexican Restaurant         Grocery Store   

  6th Most Common Venue 7th Most Common Venue 8th Most Common Venue  \
0                  Food     Fondue Restaurant           Flower Shop   
1            Food Court                  Food     Fondue Restaurant   
2                Bakery           Wings Joint    Chinese Restaurant   
2                Bakery           Wings Joint    Chinese Restaurant   
2                Bakery           Wings Joint    Chinese Restaurant   

  9th Most Common Venue     10th Most Common Venue  latitude  longitude  
0     Fish & Chips Shop       Fast Food Restaurant     45.80    -123.96  
1           Flower Shop          Fish & Chips Shop     45.23    -122.80  
2      Sushi Restaurant  Middle Eastern Restaurant     45.49    -122.80  
2      Sushi Restaurant  Middle Eastern Restaurant     45.49    -122.80  
2      Sushi Restaurant  Middle Eastern Restaurant     45.49    -122.80

In [35]:
map_clustersCA = folium.Map(location=[33.4934, -117.14879], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]



# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted['latitude'], neighborhoods_venues_sorted['longitude'], \
                                  neighborhoods_venues_sorted['Neighborhood'], neighborhoods_venues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clustersCA)
       
map_clustersCA

In [36]:
map_clustersOR = folium.Map(location=[45.51179, -122.67563], zoom_start=11)

# set color scheme for the clusters
# x = np.arange(kclusters)
# ys = [i + x + (i*x)**2 for i in range(kclusters)]
# colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
# rainbow = [colors.rgb2hex(i) for i in colors_array]

# temecula_merged.rename(columns = {'primary_city':'Neighborhood'}, inplace = True)

# add markers to the map
markers_colors2 = []
for lat, lon, poi, cluster in zip(neighborhoods_venues_sorted1['latitude'], neighborhoods_venues_sorted1['longitude'], \
                                  neighborhoods_venues_sorted1['Neighborhood'], neighborhoods_venues_sorted1['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clustersOR)
       
map_clustersOR

In [88]:
filter1 = neighborhoods_venues_sorted1['1st Most Common Venue'].str.contains('Restaurant') | neighborhoods_venues_sorted1['1st Most Common Venue'].str.contains('Joint')
filter2 = neighborhoods_venues_sorted1['2nd Most Common Venue'].str.contains('Restaurant') | neighborhoods_venues_sorted1['2nd Most Common Venue'].str.contains('Joint')
filter3 = neighborhoods_venues_sorted1['1st Most Common Venue'].str.contains('Restaurant') | neighborhoods_venues_sorted1['1st Most Common Venue'].str.contains('Joint')
docs_OR = neighborhoods_venues_sorted1.where(filter1 | filter2 | filter3)
docs_OR.dropna()


Cluster Labels  Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
2              0.0     Beaverton     Korean Restaurant           Coffee Shop   
2              0.0     Beaverton     Korean Restaurant           Coffee Shop   
2              0.0     Beaverton     Korean Restaurant           Coffee Shop   
2              0.0     Beaverton     Korean Restaurant           Coffee Shop   
3              2.0   Bridal Veil   American Restaurant           Yoga Studio   
5              0.0  Cannon Beach   American Restaurant            Restaurant   
18             0.0    Marylhurst    Chinese Restaurant      Ramen Restaurant   
19             2.0        Mehama         Deli / Bodega   American Restaurant   

          3rd Most Common Venue 4th Most Common Venue 5th Most Common Venue  \
2                Sandwich Place    Mexican Restaurant         Grocery Store   
2                Sandwich Place    Mexican Restaurant         Grocery Store   
2                Sandwich Place    Mexican Restaurant         Grocery Store   
2                Sandwich Place    Mexican Restaurant         Grocery Store   
3   Eastern European Restaurant                Forest            Food Truck   
5              Toy / Game Store           Art Gallery                Bakery   
18            Martial Arts Dojo          Optical Shop    Mexican Restaurant   
19                  Yoga Studio     Electronics Store                Forest   

   6th Most Common Venue   7th Most Common Venue 8th Most Common Venue  \
2                 Bakery             Wings Joint    Chinese Restaurant   
2                 Bakery             Wings Joint    Chinese Restaurant   
2                 Bakery             Wings Joint    Chinese Restaurant   
2                 Bakery             Wings Joint    Chinese Restaurant   
3             Food Court                    Food     Fondue Restaurant   
5                   Café  Furniture / Home Store           Candy Store   
18           Coffee Shop           Grocery Store                   Gym   
19            Food Truck              Food Court                  Food   

   9th Most Common Venue     10th Most Common Venue  latitude  longitude  
2       Sushi Restaurant  Middle Eastern Restaurant     45.49    -122.80  
2       Sushi Restaurant  Middle Eastern Restaurant     45.49    -122.80  
2       Sushi Restaurant  Middle Eastern Restaurant     45.49    -122.80  
2       Sushi Restaurant  Middle Eastern Restaurant     45.48    -122.80  
3            Flower Shop          Fish & Chips Shop     45.58    -122.11  
5     Seafood Restaurant                Pizza Place     45.90    -123.96  
18        Sandwich Place        Japanese Restaurant     45.39    -122.64  
19     Fondue Restaurant                Flower Shop     44.79    -122.62

### Orgeon's table suggests that I would prefer to live in Beaverton, OR rather than Portland proper.  Southern CA is looking a bit more appealing.

In [63]:
filter1a = neighborhoods_venues_sorted['3rd Most Common Venue'].str.contains('Restaurant') | neighborhoods_venues_sorted['3rd Most Common Venue'].str.contains('Joint')
filter2a = neighborhoods_venues_sorted['2nd Most Common Venue'].str.contains('Restaurant') | neighborhoods_venues_sorted['2nd Most Common Venue'].str.contains('Joint')
filter3a = neighborhoods_venues_sorted['1st Most Common Venue'].str.contains('Restaurant') | neighborhoods_venues_sorted['1st Most Common Venue'].str.contains('Joint')
docs_CA = neighborhoods_venues_sorted.where(filter1a | filter2a | filter3a)
docs_CA.dropna()

Cluster Labels            Neighborhood 1st Most Common Venue  \
3              3.0                Homeland          Home Service   
4              4.0               Idyllwild                   Spa   
5              1.0           Lake Elsinore        Sandwich Place   
5              1.0           Lake Elsinore        Sandwich Place   
5              1.0           Lake Elsinore        Sandwich Place   
6              1.0  March Air Reserve Base           Coffee Shop   
8              0.0               Mira Loma                  Food   
9              1.0           Moreno Valley    Mexican Restaurant   
9              1.0           Moreno Valley    Mexican Restaurant   
9              1.0           Moreno Valley    Mexican Restaurant   
9              1.0           Moreno Valley    Mexican Restaurant   
9              1.0           Moreno Valley    Mexican Restaurant   
9              1.0           Moreno Valley    Mexican Restaurant   
9              1.0           Moreno Valley    Mexican Restaurant   
10             1.0                Murrieta    Mexican Restaurant   
10             1.0                Murrieta    Mexican Restaurant   
10             1.0                Murrieta    Mexican Restaurant   
11             1.0                  Perris                  Park   
11             1.0                  Perris                  Park   
11             1.0                  Perris                  Park   
11             1.0                  Perris                  Park   
12             1.0               Riverside        Clothing Store   
12             1.0               Riverside        Clothing Store   
12             1.0               Riverside        Clothing Store   
12             1.0               Riverside        Clothing Store   
12             1.0               Riverside        Clothing Store   
12             1.0               Riverside        Clothing Store   
12             1.0               Riverside        Clothing Store   
12             1.0               Riverside        Clothing Store   
12             1.0               Riverside        Clothing Store   
12             1.0               Riverside        Clothing Store   
15             1.0                Temecula                 Hotel   
15             1.0                Temecula                 Hotel   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
3          Hot Dog Joint        Cosmetics Shop         Deli / Bodega   
4          Women's Store   Fried Chicken Joint        Cosmetics Shop   
5   Fast Food Restaurant           Pizza Place   Fried Chicken Joint   
5   Fast Food Restaurant           Pizza Place   Fried Chicken Joint   
5   Fast Food Restaurant           Pizza Place   Fried Chicken Joint   
6   Fast Food Restaurant         Women's Store     Convenience Store   
8          Women's Store         Hot Dog Joint        Cosmetics Shop   
9    Fried Chicken Joint        Ice Cream Shop   Filipino Restaurant   
9    Fried Chicken Joint        Ice Cream Shop   Filipino Restaurant   
9    Fried Chicken Joint        Ice Cream Shop   Filipino Restaurant   
9    Fried Chicken Joint        Ice Cream Shop   Filipino Restaurant   
9    Fried Chicken Joint        Ice Cream Shop   Filipino Restaurant   
9    Fried Chicken Joint        Ice Cream Shop   Filipino Restaurant   
9    Fried Chicken Joint        Ice Cream Shop   Filipino Restaurant   
10           Music Venue          Dessert Shop                Office   
10           Music Venue          Dessert Shop                Office   
10           Music Venue          Dessert Shop                Office   
11           Pizza Place    Mexican Restaurant  Fast Food Restaurant   
11           Pizza Place    Mexican Restaurant  Fast Food Restaurant   
11           Pizza Place    Mexican Restaurant  Fast Food Restaurant   
11           Pizza Place    Mexican Restaurant  Fast Food Restaurant   
12  Fast Food Restaurant   American Restaurant    Mexican Restaurant   
12  Fast Food Restaurant   American

### The two tables above suggest that for a food-lover, Temecula is defintely worth digging deeper.

### Let's evaluate it from a cultural perspective.

In [96]:
df_CA_arts = temecula_venues[temecula_venues['Venue Category'].str.contains('Museum')]
df_CA_arts1 = temecula_venues[temecula_venues['Venue Category'].str.contains('Perform')]
df_CA_arts.append(df_CA_arts1)

Neighborhood  Neighborhood Latitude  Neighborhood Longitude  \
1        Banning                  33.95                 -116.83   
7      Riverside                  33.98                 -117.37   
17     Riverside                  33.98                 -117.37   
28     Riverside                  33.98                 -117.37   
320     Temecula                  33.50                 -117.15   
3      Riverside                  33.98                 -117.37   
13     Riverside                  33.98                 -117.37   
33     Riverside                  33.98                 -117.37   

                                    Venue  Venue Latitude  Venue Longitude  \
1                            Malki Museum       33.949280      -116.825279   
7              Riverside Art Museum (RAM)       33.981682      -117.370581   
17   UCR California Museum Of Photography       33.981360      -117.374530   
28          Riverside Metropolitan Museum       33.982215      -117.372374   
320                Temecula Valley Museum       33.497441      -117.150993   
3          Riverside Municipal Auditorium       33.982062      -117.370789   
13         Riverside Life And Arts Center       33.981170      -117.371643   
33              Culver Center of the Arts       33.981375      -117.374569   

            Venue Category  
1                   Museum  
7               Art Museum  
17              Art Museum  
28                  Museum  
320         History Museum  
3    Performing Arts Venue  
13   Performing Arts Venue  
33   Performing Arts Venue

### Much as the Portland data suggested I live in Beaverton, the Temecula data suggests I should live in Riverside as the food and cultural scene are more prominent in that area.

In [97]:
# create map of Temecula using latitude and longitude values

map_riverside = folium.Map(location=[33.946484, -117.38317], zoom_start=10)

# add markers to map
for lat, lng, primCity, popul in zip(df_CA['latitude'], df_CA['longitude'], df_CA['primary_city'], df_CA['irs_estimated_population_2015']):
    label = '{}, {}'.format(primCity, popul)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_riverside)  
    
map_riverside

### My new home!  Riverside, CA here I come!